In [ ]:
from typing import List

import numpy as np

# ref: https://sungkee-book.tistory.com/13


In [ ]:

def rss(y_true: List[int], y_pred: List[int]):
    rss = sum((true - pred) ** 2 for true, pred in zip(y_true, y_pred))
    return rss 

def rss_np(y_true, y_pred):
    """
    Why doesn't have support for numpy.typing.NDArray?
    :param y_true: 
    :param y_pred: 
    :return: 
    """
    residuals = y_true - y_pred 
    rss = np.sum(residuals ** 2)
    return rss 

In [0]:


def als_without_numpy(R, K, max_iter=10, lambda_=0.1):
    # R: 사용자-아이템 평가 행렬
    # K: 잠재 요인의 수
    # max_iter: 최대 반복 횟수
    # lambda_: 정규화 파라미터

    m, n = len(R), len(R[0])
    X = [[0.5 for _ in range(K)] for _ in range(m)]
    Y = [[0.5 for _ in range(K)] for _ in range(n)]

    def product(X, Y):
        result = [[sum(x * y for x, y in zip(X_row, Y_col)) for Y_col in zip(*Y)] for X_row in X]
        return result

    for iteration in range(max_iter):
        # X 고정, Y 최적화
        for i in range(m):
            for k in range(K):
                numerator = sum(R[i][j] * Y[j][k] for j in range(n))
                denominator = lambda_ + sum(Y[j][k] ** 2 for j in range(n))
                X[i][k] = numerator / denominator

        # Y 고정, X 최적화
        for j in range(n):
            for k in range(K):
                numerator = sum(R[i][j] * X[i][k] for i in range(m))
                denominator = lambda_ + sum(X[i][k] ** 2 for i in range(m))
                Y[j][k] = numerator / denominator

    return product(X, Y)

# 예제 데이터
R = [
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
]

# ALS 실행
K = 2
approx_R = als_without_numpy(R, K)

for row in approx_R:
    print(row)


In [5]:
import numpy as np

def als_with_numpy(R, K, max_iter=10, lambda_=0.1):
    m, n = R.shape
    X = np.random.rand(m, K)
    Y = np.random.rand(n, K)

    for iteration in range(max_iter):
        # X 고정, Y 최적화
        for i in range(m):
            # np.linalg.solve(A, B): A * x = B를 만족하는 x
            # np.eye(K): K x K 단위 행렬(identity matrix)을 생성
            # np.dot(Y.T, Y) : K x K
            X[i] = np.linalg.solve(np.dot(Y.T, Y) + lambda_ * np.eye(K), 
                                   np.dot(Y.T, R[i]))

        # Y 고정, X 최적화
        for j in range(n):
            Y[j] = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(K), 
                                   np.dot(X.T, R[:, j]))

    return np.dot(X, Y.T) # m x n

# 예제 데이터
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

# ALS 실행
K = 2
approx_R = als_with_numpy(R, K)

print(approx_R)



[[ 5.10563127  1.8964268  -0.71398157  1.55937345]
 [ 3.41419075  1.27426338 -0.45139675  1.08816549]
 [ 1.54374068  1.04013781  1.77658274  3.94356916]
 [ 1.17234583  0.79988955  1.39181411  3.06913192]
 [-0.44141961  0.54210469  3.07588545  5.11766672]]
